In [1]:
import urllib
import urllib.parse
import json
import pandas as pd
from datetime import datetime
import os


##################################
# function to unnest json for each month
def extract_monthly_data(data):
    num_days = len(data)
    # initialize df_month to store return data
    df_month = pd.DataFrame()
    for i in range(num_days):
        # extract this day
        d = data[i]
        # astronomy data is the same for the whole day
        astr_df = pd.DataFrame(d['astronomy'])
        # hourly data; temperature for each hour of the day
        hourly_df = pd.DataFrame(d['hourly'])
        # this wanted_key will be duplicated and use 'ffill' to fill up the NAs
        wanted_keys = ['date', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex']  # The keys you want
        subset_d = dict((k, d[k]) for k in wanted_keys if k in d)
        this_df = pd.DataFrame(subset_d, index=[0])
        df = pd.concat([this_df.reset_index(drop=True), astr_df], axis=1)
        # concat selected astonomy columns with hourly data
        df = pd.concat([df, hourly_df], axis=1)
        df = df.fillna(method='ffill')
        # make date_time columm to proper format
        # fill leading zero for hours to 4 digits (0000-2400 hr)
        df['time'] = df['time'].apply(lambda x: x.zfill(4))
        # keep only first 2 digit (00-24 hr) 
        df['time'] = df['time'].str[:2]
        # convert to pandas datetime
        df['date_time'] = pd.to_datetime(df['date'] + ' ' + df['time'])
        # keep only interested columns
        col_to_keep = ['date_time', 'maxtempC', 'mintempC', 'totalSnow_cm', 'sunHour', 'uvIndex',
                       'moon_illumination', 'moonrise', 'moonset', 'sunrise', 'sunset',
                       'DewPointC', 'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph',
                       'cloudcover', 'humidity', 'precipMM', 'pressure', 'tempC', 'visibility',
                       'winddirDegree', 'windspeedKmph']
        df = df[col_to_keep]
        df = df.loc[:,~df.columns.duplicated()]
        df_month = pd.concat([df_month, df])
    return (df_month)


##################################
# function to retrive data by date range and location
# default frequency = 1 hr
# each month costs 1 request (free trial 500 requests/key, as of 30-May-2019)
def retrieve_this_location(api_key, location, start_date, end_date, frequency, response_cache_path):
    start_time = datetime.now()

    # create list of first day of month for range between start and end dates non-inclusive (open)
    list_mon_begin = pd.date_range(start_date, end_date, freq='MS', closed='right')
    # convert to Series and add start_date at beginning
    list_mon_begin = pd.concat([pd.Series(pd.to_datetime(start_date)), pd.Series(list_mon_begin)], ignore_index=True)

    # create list of month end dates for range between start and end dates non-inclusive (open)
    list_mon_end = pd.date_range(start_date, end_date, freq='M', closed='left')
    # convert to Series and add end_date at end
    list_mon_end = pd.concat([pd.Series(list_mon_end), pd.Series(pd.to_datetime(end_date))], ignore_index=True)

    # count number of months to be retrieved
    total_months = len(list_mon_begin)

    # initialize df_hist to store return data
    df_hist = pd.DataFrame()
    for m in range(total_months):
        start_d = str(list_mon_begin[m])[:10]
        end_d = str(list_mon_end[m])[:10]
        file_path = f'{response_cache_path}/{location}_{start_d}_{end_d}'
        if response_cache_path and os.path.exists(file_path):
            print('Reading cached data for ' + location + ': from ' + start_d + ' to ' + end_d)
            with open(f'{response_cache_path}/{location}_{start_d}_{end_d}', 'r') as f:
                json_data = json.load(f)
        else:
            print('Currently retrieving data for ' + location + ': from ' + start_d + ' to ' + end_d)
            url_page = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=' + api_key + '&q=' + location + '&format=json&date=' + start_d + '&enddate=' + end_d + '&tp=' + str(
                frequency)
            json_page = urllib.request.urlopen(url_page, timeout=10)
            json_data = json.loads(json_page.read().decode())

        if response_cache_path:
            with open(f'{response_cache_path}/{location}_{start_d}_{end_d}', 'w') as f:
                json.dump(json_data, f)
        data = json_data['data']['weather']
        # call function to extract json object
        df_this_month = extract_monthly_data(data)
        df_this_month['location'] = location
        df_hist = pd.concat([df_hist, df_this_month])

        time_elapsed = datetime.now() - start_time
        print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))
    return (df_hist)


##################################
# main function to retrive the data by location list
def retrieve_hist_data(api_key, location_list, start_date, end_date, frequency, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None):
    result_list = []
    for location in location_list:
        print('\n\nRetrieving weather data for ' + location + '\n\n')
        df_this_city = retrieve_this_location(api_key, location, start_date, end_date, frequency, response_cache_path)

        if (location_label == True):
            # add city name as prefix to the colnames
            df_this_city = df_this_city.add_prefix(location + '_')
            df_this_city.columns.values[0] = 'date_time'

        if (export_csv == True):
            df_this_city.to_csv('./' + location + '.csv', header=True, index=False)
            print('\n\nexport ' + location + ' completed!\n\n')

        if (store_df == True):
            # save result as object in the work space
            result_list.append(df_this_city)

    return (result_list)
##################################

In [2]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.575694,-79.337436"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.575694,-79.337436


Currently retrieving data for 37.575694,-79.337436: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.433237
Currently retrieving data for 37.575694,-79.337436: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.682618
Currently retrieving data for 37.575694,-79.337436: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:03.871797
Currently retrieving data for 37.575694,-79.337436: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.014142
Currently retrieving data for 37.575694,-79.337436: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.392487
Currently retrieving data for 37.575694,-79.337436: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:07.580116
Currently retrieving data for 37.575694,-79.337436: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.085636
Currently retrieving data for 37.575694,-79.337436: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:25.964389
Currently retrieving data for 37.575694,-79.337436: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:27.144991
Currently retrieving data for 37.575694,-79.337436: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:28.363927
Currently retrieving data for 37.575694,-79.337436: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:29.832138
Currently retrieving data for 37.575694,-79.337436: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:31.196327
Currently retrieving data for 37.575694,-79.337436: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:33.553813
Currently retrieving data for 37.575694,-79.337436: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:34.712084
Currently retrieving data for 37.575694,-79.337436: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:35.928603
Currently retrieving data for 37.575694,-79.337436: from 2017-04-01 to 2017-04-30
Time ela

In [3]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.638096,-112.167922","36.029978,-92.584468"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.638096,-112.167922


Currently retrieving data for 37.638096,-112.167922: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.448463
Currently retrieving data for 37.638096,-112.167922: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.758409
Currently retrieving data for 37.638096,-112.167922: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:03.901067
Currently retrieving data for 37.638096,-112.167922: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.015237
Currently retrieving data for 37.638096,-112.167922: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.434958
Currently retrieving data for 37.638096,-112.167922: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:07.587539
Currently retrieving data for 37.638096,-112.167922: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.533901
Currently retrieving data for 37.638096,-112.167922: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:17.205243
Currently retrieving data for 37.638096,-112.167922: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:18.348624
Currently retrieving data for 37.638096,-112.167922: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:19.401305
Currently retrieving data for 37.638096,-112.167922: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:20.443026
Currently retrieving data for 37.638096,-112.167922: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:21.528099
Currently retrieving data for 37.638096,-112.167922: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:23.543581
Currently retrieving data for 37.638096,-112.167922: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:24.739936
Currently retrieving data for 37.638096,-112.167922: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:25.809996
Currently retrieving data for 37.638096,-112.167922: from 2017-04-01 to 2017-04-30


Time elapsed (hh:mm:ss.ms) 0:00:27.064684
Currently retrieving data for 36.029978,-92.584468: from 2013-01-01 to 2013-01-31
Time elapsed (hh:mm:ss.ms) 0:00:28.502252
Currently retrieving data for 36.029978,-92.584468: from 2013-02-01 to 2013-02-28
Time elapsed (hh:mm:ss.ms) 0:00:29.585428
Currently retrieving data for 36.029978,-92.584468: from 2013-03-01 to 2013-03-31
Time elapsed (hh:mm:ss.ms) 0:00:30.686309
Currently retrieving data for 36.029978,-92.584468: from 2013-04-01 to 2013-04-30
Time elapsed (hh:mm:ss.ms) 0:00:31.777608
Currently retrieving data for 36.029978,-92.584468: from 2013-05-01 to 2013-05-31
Time elapsed (hh:mm:ss.ms) 0:00:32.879662
Currently retrieving data for 36.029978,-92.584468: from 2013-06-01 to 2013-06-30
Time elapsed (hh:mm:ss.ms) 0:00:33.942782
Currently retrieving data for 36.029978,-92.584468: from 2013-07-01 to 2013-07-31
Time elapsed (hh:mm:ss.ms) 0:00:35.046927
Currently retrieving data for 36.029978,-92.584468: from 2013-08-01 to 2013-08-31
Time ela

Time elapsed (hh:mm:ss.ms) 0:01:44.036337
Currently retrieving data for 36.029978,-92.584468: from 2018-08-01 to 2018-08-31
Time elapsed (hh:mm:ss.ms) 0:01:45.082031
Currently retrieving data for 36.029978,-92.584468: from 2018-09-01 to 2018-09-30
Time elapsed (hh:mm:ss.ms) 0:01:46.101911
Currently retrieving data for 36.029978,-92.584468: from 2018-10-01 to 2018-10-31
Time elapsed (hh:mm:ss.ms) 0:01:47.140757
Currently retrieving data for 36.029978,-92.584468: from 2018-11-01 to 2018-11-30
Time elapsed (hh:mm:ss.ms) 0:01:48.162596
Currently retrieving data for 36.029978,-92.584468: from 2018-12-01 to 2018-12-31
Time elapsed (hh:mm:ss.ms) 0:01:49.236032
Currently retrieving data for 36.029978,-92.584468: from 2019-01-01 to 2019-01-31
Time elapsed (hh:mm:ss.ms) 0:01:50.471854
Currently retrieving data for 36.029978,-92.584468: from 2019-02-01 to 2019-02-28
Time elapsed (hh:mm:ss.ms) 0:01:51.518165
Currently retrieving data for 36.029978,-92.584468: from 2019-03-01 to 2019-03-31
Time ela

In [12]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["36.029978,-92.584468"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 36.029978,-92.584468


Currently retrieving data for 36.029978,-92.584468: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.559089
Currently retrieving data for 36.029978,-92.584468: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.116178
Currently retrieving data for 36.029978,-92.584468: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.483256
Currently retrieving data for 36.029978,-92.584468: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.796332
Currently retrieving data for 36.029978,-92.584468: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.241414
Currently retrieving data for 36.029978,-92.584468: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.604492
Currently retrieving data for 36.029978,-92.584468: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.922568
Currently retrieving data for 36.029978,-92.584468: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:36.842539
Currently retrieving data for 36.029978,-92.584468: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:38.125612
Currently retrieving data for 36.029978,-92.584468: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:39.390685
Currently retrieving data for 36.029978,-92.584468: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:40.624755
Currently retrieving data for 36.029978,-92.584468: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:41.978833
Currently retrieving data for 36.029978,-92.584468: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:43.586925
Currently retrieving data for 36.029978,-92.584468: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:44.892000
Currently retrieving data for 36.029978,-92.584468: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:46.153072
Currently retrieving data for 36.029978,-92.584468: from 2017-04-01 to 2017-04-30
Time ela

In [13]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["43.389240,-4.332760"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 43.389240,-4.332760


Currently retrieving data for 43.389240,-4.332760: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.999115
Currently retrieving data for 43.389240,-4.332760: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.225242
Currently retrieving data for 43.389240,-4.332760: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.114350
Currently retrieving data for 43.389240,-4.332760: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:08.200469
Currently retrieving data for 43.389240,-4.332760: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:10.063576
Currently retrieving data for 43.389240,-4.332760: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:12.073691
Currently retrieving data for 43.389240,-4.332760: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:13.656781
Currently retrieving data for 43.389240,-4.332760: from 2011-10-01 to 2011-10-31
Time e

Time elapsed (hh:mm:ss.ms) 0:02:13.830655
Currently retrieving data for 43.389240,-4.332760: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:02:15.584755
Currently retrieving data for 43.389240,-4.332760: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:02:17.198848
Currently retrieving data for 43.389240,-4.332760: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:02:19.502979
Currently retrieving data for 43.389240,-4.332760: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:02:21.710106
Currently retrieving data for 43.389240,-4.332760: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:23.101185
Currently retrieving data for 43.389240,-4.332760: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:24.459263
Currently retrieving data for 43.389240,-4.332760: from 2017-04-01 to 2017-04-30
Time elapsed (hh:mm:ss.ms) 0:02:25.751337
Currently retrieving data for 43.389240,-4.332760: from 2017-05-01 to 2017-05-31
Time elapsed (hh

In [14]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["35.237075,-75.902467"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 35.237075,-75.902467


Currently retrieving data for 35.237075,-75.902467: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.945111
Currently retrieving data for 35.237075,-75.902467: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.710212
Currently retrieving data for 35.237075,-75.902467: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.495314
Currently retrieving data for 35.237075,-75.902467: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.084405
Currently retrieving data for 35.237075,-75.902467: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.699497
Currently retrieving data for 35.237075,-75.902467: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:12.263701
Currently retrieving data for 35.237075,-75.902467: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:13.968798
Currently retrieving data for 35.237075,-75.902467: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:51.428373
Currently retrieving data for 35.237075,-75.902467: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:52.801451
Currently retrieving data for 35.237075,-75.902467: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:54.298537
Currently retrieving data for 35.237075,-75.902467: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:55.881628
Currently retrieving data for 35.237075,-75.902467: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:57.326710
Currently retrieving data for 35.237075,-75.902467: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:59.260821
Currently retrieving data for 35.237075,-75.902467: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:01.479948
Currently retrieving data for 35.237075,-75.902467: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:02.833025
Currently retrieving data for 35.237075,-75.902467: from 2017-04-01 to 2017-04-30
Time ela

In [15]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["28.799910,-80.806290"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 28.799910,-80.806290


Currently retrieving data for 28.799910,-80.806290: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.326076
Currently retrieving data for 28.799910,-80.806290: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.930168
Currently retrieving data for 28.799910,-80.806290: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.579262
Currently retrieving data for 28.799910,-80.806290: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.153352
Currently retrieving data for 28.799910,-80.806290: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.745443
Currently retrieving data for 28.799910,-80.806290: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.785560
Currently retrieving data for 28.799910,-80.806290: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.589663
Currently retrieving data for 28.799910,-80.806290: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:50.467318
Currently retrieving data for 28.799910,-80.806290: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:52.046409
Currently retrieving data for 28.799910,-80.806290: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:53.518493
Currently retrieving data for 28.799910,-80.806290: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:54.950575
Currently retrieving data for 28.799910,-80.806290: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:56.391657
Currently retrieving data for 28.799910,-80.806290: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:58.270765
Currently retrieving data for 28.799910,-80.806290: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:00.298881
Currently retrieving data for 28.799910,-80.806290: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:01.703961
Currently retrieving data for 28.799910,-80.806290: from 2017-04-01 to 2017-04-30
Time ela

In [16]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["38.148083,-109.797759"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 38.148083,-109.797759


Currently retrieving data for 38.148083,-109.797759: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.369079
Currently retrieving data for 38.148083,-109.797759: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.764158
Currently retrieving data for 38.148083,-109.797759: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.110235
Currently retrieving data for 38.148083,-109.797759: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.368307
Currently retrieving data for 38.148083,-109.797759: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.654381
Currently retrieving data for 38.148083,-109.797759: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.955512
Currently retrieving data for 38.148083,-109.797759: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.227585
Currently retrieving data for 38.148083,-109.797759: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:33.082324
Currently retrieving data for 38.148083,-109.797759: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:34.346397
Currently retrieving data for 38.148083,-109.797759: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:35.768478
Currently retrieving data for 38.148083,-109.797759: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:37.067552
Currently retrieving data for 38.148083,-109.797759: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:38.371627
Currently retrieving data for 38.148083,-109.797759: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:40.589754
Currently retrieving data for 38.148083,-109.797759: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.014835
Currently retrieving data for 38.148083,-109.797759: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:43.315910
Currently retrieving data for 38.148083,-109.797759: from 2017-04-01 to 2017-04-30


In [17]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["39.154500,-108.730190"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 39.154500,-108.730190


Currently retrieving data for 39.154500,-108.730190: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.386079
Currently retrieving data for 39.154500,-108.730190: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.893165
Currently retrieving data for 39.154500,-108.730190: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.303246
Currently retrieving data for 39.154500,-108.730190: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.575319
Currently retrieving data for 39.154500,-108.730190: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.900394
Currently retrieving data for 39.154500,-108.730190: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.206469
Currently retrieving data for 39.154500,-108.730190: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.452540
Currently retrieving data for 39.154500,-108.730190: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:33.018320
Currently retrieving data for 39.154500,-108.730190: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:34.269392
Currently retrieving data for 39.154500,-108.730190: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:35.623469
Currently retrieving data for 39.154500,-108.730190: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.957545
Currently retrieving data for 39.154500,-108.730190: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:38.338624
Currently retrieving data for 39.154500,-108.730190: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:40.252734
Currently retrieving data for 39.154500,-108.730190: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:41.880827
Currently retrieving data for 39.154500,-108.730190: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:43.214903
Currently retrieving data for 39.154500,-108.730190: from 2017-04-01 to 2017-04-30


In [18]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["39.655829,-77.487248"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 39.655829,-77.487248


Currently retrieving data for 39.655829,-77.487248: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.404080
Currently retrieving data for 39.655829,-77.487248: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.887165
Currently retrieving data for 39.655829,-77.487248: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.191240
Currently retrieving data for 39.655829,-77.487248: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.520316
Currently retrieving data for 39.655829,-77.487248: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.827390
Currently retrieving data for 39.655829,-77.487248: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.095463
Currently retrieving data for 39.655829,-77.487248: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.400538
Currently retrieving data for 39.655829,-77.487248: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:29.438115
Currently retrieving data for 39.655829,-77.487248: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:30.775192
Currently retrieving data for 39.655829,-77.487248: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:32.236276
Currently retrieving data for 39.655829,-77.487248: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:33.606354
Currently retrieving data for 39.655829,-77.487248: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:35.055437
Currently retrieving data for 39.655829,-77.487248: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:36.946545
Currently retrieving data for 39.655829,-77.487248: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:38.255620
Currently retrieving data for 39.655829,-77.487248: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:39.571695
Currently retrieving data for 39.655829,-77.487248: from 2017-04-01 to 2017-04-30
Time ela

In [19]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.610439,-112.831020"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.610439,-112.831020


Currently retrieving data for 37.610439,-112.831020: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.531088
Currently retrieving data for 37.610439,-112.831020: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.078176
Currently retrieving data for 37.610439,-112.831020: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.463255
Currently retrieving data for 37.610439,-112.831020: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.917339
Currently retrieving data for 37.610439,-112.831020: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.233414
Currently retrieving data for 37.610439,-112.831020: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.850506
Currently retrieving data for 37.610439,-112.831020: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.132580
Currently retrieving data for 37.610439,-112.831020: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:29.821138
Currently retrieving data for 37.610439,-112.831020: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:31.086210
Currently retrieving data for 37.610439,-112.831020: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:32.479290
Currently retrieving data for 37.610439,-112.831020: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:33.920372
Currently retrieving data for 37.610439,-112.831020: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:35.459460
Currently retrieving data for 37.610439,-112.831020: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:37.189559
Currently retrieving data for 37.610439,-112.831020: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:38.457632
Currently retrieving data for 37.610439,-112.831020: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:39.741705
Currently retrieving data for 37.610439,-112.831020: from 2017-04-01 to 2017-04-30


In [20]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["34.503990,-96.949650"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 34.503990,-96.949650


Currently retrieving data for 34.503990,-96.949650: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.626093
Currently retrieving data for 34.503990,-96.949650: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.076176
Currently retrieving data for 34.503990,-96.949650: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.445254
Currently retrieving data for 34.503990,-96.949650: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.709327
Currently retrieving data for 34.503990,-96.949650: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.083405
Currently retrieving data for 34.503990,-96.949650: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.390480
Currently retrieving data for 34.503990,-96.949650: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.661553
Currently retrieving data for 34.503990,-96.949650: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:37.663586
Currently retrieving data for 34.503990,-96.949650: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:39.033665
Currently retrieving data for 34.503990,-96.949650: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:40.339739
Currently retrieving data for 34.503990,-96.949650: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:41.605812
Currently retrieving data for 34.503990,-96.949650: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:42.943888
Currently retrieving data for 34.503990,-96.949650: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:44.878999
Currently retrieving data for 34.503990,-96.949650: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:46.198074
Currently retrieving data for 34.503990,-96.949650: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:47.593154
Currently retrieving data for 34.503990,-96.949650: from 2017-04-01 to 2017-04-30
Time ela

In [21]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["32.011248,-109.354945"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 32.011248,-109.354945


Currently retrieving data for 32.011248,-109.354945: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.404080
Currently retrieving data for 32.011248,-109.354945: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.069175
Currently retrieving data for 32.011248,-109.354945: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.398251
Currently retrieving data for 32.011248,-109.354945: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.686325
Currently retrieving data for 32.011248,-109.354945: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.015401
Currently retrieving data for 32.011248,-109.354945: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.299474
Currently retrieving data for 32.011248,-109.354945: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.663552
Currently retrieving data for 32.011248,-109.354945: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:31.334224
Currently retrieving data for 32.011248,-109.354945: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.553294
Currently retrieving data for 32.011248,-109.354945: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:33.857368
Currently retrieving data for 32.011248,-109.354945: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.157442
Currently retrieving data for 32.011248,-109.354945: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.636527
Currently retrieving data for 32.011248,-109.354945: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.418629
Currently retrieving data for 32.011248,-109.354945: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:39.719703
Currently retrieving data for 32.011248,-109.354945: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.005777
Currently retrieving data for 32.011248,-109.354945: from 2017-04-01 to 2017-04-30


In [22]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["34.014210,-120.349007"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 34.014210,-120.349007


Currently retrieving data for 34.014210,-120.349007: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.570090
Currently retrieving data for 34.014210,-120.349007: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.029174
Currently retrieving data for 34.014210,-120.349007: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.315247
Currently retrieving data for 34.014210,-120.349007: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.610321
Currently retrieving data for 34.014210,-120.349007: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.859393
Currently retrieving data for 34.014210,-120.349007: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.182468
Currently retrieving data for 34.014210,-120.349007: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.482543
Currently retrieving data for 34.014210,-120.349007: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:31.715246
Currently retrieving data for 34.014210,-120.349007: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.971318
Currently retrieving data for 34.014210,-120.349007: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.261392
Currently retrieving data for 34.014210,-120.349007: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.814481
Currently retrieving data for 34.014210,-120.349007: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.114555
Currently retrieving data for 34.014210,-120.349007: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.702646
Currently retrieving data for 34.014210,-120.349007: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:39.971718
Currently retrieving data for 34.014210,-120.349007: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.275793
Currently retrieving data for 34.014210,-120.349007: from 2017-04-01 to 2017-04-30


In [23]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["39.562141,-77.853069"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 39.562141,-77.853069


Currently retrieving data for 39.562141,-77.853069: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.449083
Currently retrieving data for 39.562141,-77.853069: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.069175
Currently retrieving data for 39.562141,-77.853069: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.419252
Currently retrieving data for 39.562141,-77.853069: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.713326
Currently retrieving data for 39.562141,-77.853069: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.018401
Currently retrieving data for 39.562141,-77.853069: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.343477
Currently retrieving data for 39.562141,-77.853069: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.665552
Currently retrieving data for 39.562141,-77.853069: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:34.442401
Currently retrieving data for 39.562141,-77.853069: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.718474
Currently retrieving data for 39.562141,-77.853069: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:37.056551
Currently retrieving data for 39.562141,-77.853069: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:38.339624
Currently retrieving data for 39.562141,-77.853069: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:39.790707
Currently retrieving data for 39.562141,-77.853069: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:41.522806
Currently retrieving data for 39.562141,-77.853069: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.786879
Currently retrieving data for 39.562141,-77.853069: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:44.114955
Currently retrieving data for 39.562141,-77.853069: from 2017-04-01 to 2017-04-30
Time ela

In [24]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["33.065408,-107.495734"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 33.065408,-107.495734


Currently retrieving data for 33.065408,-107.495734: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.407080
Currently retrieving data for 33.065408,-107.495734: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.173239
Currently retrieving data for 33.065408,-107.495734: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.430311
Currently retrieving data for 33.065408,-107.495734: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.792388
Currently retrieving data for 33.065408,-107.495734: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.970513
Currently retrieving data for 33.065408,-107.495734: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:10.305589
Currently retrieving data for 33.065408,-107.495734: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.631665
Currently retrieving data for 33.065408,-107.495734: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:33.369340
Currently retrieving data for 33.065408,-107.495734: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.534464
Currently retrieving data for 33.065408,-107.495734: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:36.841539
Currently retrieving data for 33.065408,-107.495734: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:38.060609
Currently retrieving data for 33.065408,-107.495734: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:39.334682
Currently retrieving data for 33.065408,-107.495734: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:40.988776
Currently retrieving data for 33.065408,-107.495734: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.313852
Currently retrieving data for 33.065408,-107.495734: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:43.635928
Currently retrieving data for 33.065408,-107.495734: from 2017-04-01 to 2017-04-30


In [25]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["39.104764,-108.732344"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 39.104764,-108.732344


Currently retrieving data for 39.104764,-108.732344: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.052060
Currently retrieving data for 39.104764,-108.732344: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.277130
Currently retrieving data for 39.104764,-108.732344: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:03.575204
Currently retrieving data for 39.104764,-108.732344: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:04.805275
Currently retrieving data for 39.104764,-108.732344: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.102349
Currently retrieving data for 39.104764,-108.732344: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:07.417424
Currently retrieving data for 39.104764,-108.732344: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:08.670496
Currently retrieving data for 39.104764,-108.732344: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:40.368741
Currently retrieving data for 39.104764,-108.732344: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:41.744819
Currently retrieving data for 39.104764,-108.732344: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:43.018892
Currently retrieving data for 39.104764,-108.732344: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:44.344968
Currently retrieving data for 39.104764,-108.732344: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:45.583039
Currently retrieving data for 39.104764,-108.732344: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:46.906115
Currently retrieving data for 39.104764,-108.732344: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:48.172187
Currently retrieving data for 39.104764,-108.732344: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:49.453260
Currently retrieving data for 39.104764,-108.732344: from 2017-04-01 to 2017-04-30


In [26]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["42.867185,-122.165603"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 42.867185,-122.165603


Currently retrieving data for 42.867185,-122.165603: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.545089
Currently retrieving data for 42.867185,-122.165603: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.138180
Currently retrieving data for 42.867185,-122.165603: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.442254
Currently retrieving data for 42.867185,-122.165603: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.747329
Currently retrieving data for 42.867185,-122.165603: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.049403
Currently retrieving data for 42.867185,-122.165603: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.365479
Currently retrieving data for 42.867185,-122.165603: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.631551
Currently retrieving data for 42.867185,-122.165603: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:28.644070
Currently retrieving data for 42.867185,-122.165603: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:29.821138
Currently retrieving data for 42.867185,-122.165603: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:31.155214
Currently retrieving data for 42.867185,-122.165603: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:32.459289
Currently retrieving data for 42.867185,-122.165603: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:33.734361
Currently retrieving data for 42.867185,-122.165603: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:35.512463
Currently retrieving data for 42.867185,-122.165603: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:36.773535
Currently retrieving data for 42.867185,-122.165603: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:38.066609
Currently retrieving data for 42.867185,-122.165603: from 2017-04-01 to 2017-04-30


In [27]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["43.460787,-113.558986"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 43.460787,-113.558986


Currently retrieving data for 43.460787,-113.558986: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.539088
Currently retrieving data for 43.460787,-113.558986: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.014172
Currently retrieving data for 43.460787,-113.558986: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.474256
Currently retrieving data for 43.460787,-113.558986: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.765330
Currently retrieving data for 43.460787,-113.558986: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.043403
Currently retrieving data for 43.460787,-113.558986: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.463484
Currently retrieving data for 43.460787,-113.558986: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.910567
Currently retrieving data for 43.460787,-113.558986: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:31.467232
Currently retrieving data for 43.460787,-113.558986: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.746305
Currently retrieving data for 43.460787,-113.558986: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.095382
Currently retrieving data for 43.460787,-113.558986: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.329453
Currently retrieving data for 43.460787,-113.558986: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.586524
Currently retrieving data for 43.460787,-113.558986: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.099611
Currently retrieving data for 43.460787,-113.558986: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:39.288679
Currently retrieving data for 43.460787,-113.558986: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.115784
Currently retrieving data for 43.460787,-113.558986: from 2017-04-01 to 2017-04-30


In [28]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["30.832912,-81.450805"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 30.832912,-81.450805


Currently retrieving data for 30.832912,-81.450805: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.342134
Currently retrieving data for 30.832912,-81.450805: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.530259
Currently retrieving data for 30.832912,-81.450805: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.506372
Currently retrieving data for 30.832912,-81.450805: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:08.336477
Currently retrieving data for 30.832912,-81.450805: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:10.153581
Currently retrieving data for 30.832912,-81.450805: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:11.963684
Currently retrieving data for 30.832912,-81.450805: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:13.711784
Currently retrieving data for 30.832912,-81.450805: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:02:06.293223
Currently retrieving data for 30.832912,-81.450805: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:02:08.037323
Currently retrieving data for 30.832912,-81.450805: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:02:09.707419
Currently retrieving data for 30.832912,-81.450805: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:02:11.410516
Currently retrieving data for 30.832912,-81.450805: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:02:12.986606
Currently retrieving data for 30.832912,-81.450805: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:02:14.827712
Currently retrieving data for 30.832912,-81.450805: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:16.166788
Currently retrieving data for 30.832912,-81.450805: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:17.518866
Currently retrieving data for 30.832912,-81.450805: from 2017-04-01 to 2017-04-30
Time ela

In [29]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["63.457441,-150.861200"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 63.457441,-150.861200


Currently retrieving data for 63.457441,-150.861200: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.850106
Currently retrieving data for 63.457441,-150.861200: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.289189
Currently retrieving data for 63.457441,-150.861200: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.604264
Currently retrieving data for 63.457441,-150.861200: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.930340
Currently retrieving data for 63.457441,-150.861200: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.278417
Currently retrieving data for 63.457441,-150.861200: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.552490
Currently retrieving data for 63.457441,-150.861200: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.888566
Currently retrieving data for 63.457441,-150.861200: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:34.425401
Currently retrieving data for 63.457441,-150.861200: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.707475
Currently retrieving data for 63.457441,-150.861200: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:37.045551
Currently retrieving data for 63.457441,-150.861200: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:38.312624
Currently retrieving data for 63.457441,-150.861200: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:39.596697
Currently retrieving data for 63.457441,-150.861200: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:41.552809
Currently retrieving data for 63.457441,-150.861200: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.875885
Currently retrieving data for 63.457441,-150.861200: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:44.216961
Currently retrieving data for 63.457441,-150.861200: from 2017-04-01 to 2017-04-30


In [30]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["44.590647,-104.715634"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 44.590647,-104.715634


Currently retrieving data for 44.590647,-104.715634: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.503086
Currently retrieving data for 44.590647,-104.715634: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.270187
Currently retrieving data for 44.590647,-104.715634: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.600263
Currently retrieving data for 44.590647,-104.715634: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.950340
Currently retrieving data for 44.590647,-104.715634: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.296417
Currently retrieving data for 44.590647,-104.715634: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.596492
Currently retrieving data for 44.590647,-104.715634: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.341649
Currently retrieving data for 44.590647,-104.715634: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:34.833424
Currently retrieving data for 44.590647,-104.715634: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:36.069495
Currently retrieving data for 44.590647,-104.715634: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:37.386570
Currently retrieving data for 44.590647,-104.715634: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:38.944659
Currently retrieving data for 44.590647,-104.715634: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:40.267735
Currently retrieving data for 44.590647,-104.715634: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:42.095840
Currently retrieving data for 44.590647,-104.715634: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:43.706932
Currently retrieving data for 44.590647,-104.715634: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:45.119013
Currently retrieving data for 44.590647,-104.715634: from 2017-04-01 to 2017-04-30


In [31]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["36.463341,-116.869687"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 36.463341,-116.869687


Currently retrieving data for 36.463341,-116.869687: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.588091
Currently retrieving data for 36.463341,-116.869687: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.222184
Currently retrieving data for 36.463341,-116.869687: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.550260
Currently retrieving data for 36.463341,-116.869687: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.851335
Currently retrieving data for 36.463341,-116.869687: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.181411
Currently retrieving data for 36.463341,-116.869687: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.588491
Currently retrieving data for 36.463341,-116.869687: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.875565
Currently retrieving data for 36.463341,-116.869687: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:33.698359
Currently retrieving data for 36.463341,-116.869687: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.029435
Currently retrieving data for 36.463341,-116.869687: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:36.369512
Currently retrieving data for 36.463341,-116.869687: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:37.686587
Currently retrieving data for 36.463341,-116.869687: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:39.015663
Currently retrieving data for 36.463341,-116.869687: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:40.960775
Currently retrieving data for 36.463341,-116.869687: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.598868
Currently retrieving data for 36.463341,-116.869687: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:44.163958
Currently retrieving data for 36.463341,-116.869687: from 2017-04-01 to 2017-04-30


In [32]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["40.447666,-108.513720"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 40.447666,-108.513720


Currently retrieving data for 40.447666,-108.513720: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.427081
Currently retrieving data for 40.447666,-108.513720: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.050174
Currently retrieving data for 40.447666,-108.513720: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.420252
Currently retrieving data for 40.447666,-108.513720: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.799331
Currently retrieving data for 40.447666,-108.513720: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.339419
Currently retrieving data for 40.447666,-108.513720: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.832505
Currently retrieving data for 40.447666,-108.513720: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.159581
Currently retrieving data for 40.447666,-108.513720: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:39.453688
Currently retrieving data for 40.447666,-108.513720: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:40.708760
Currently retrieving data for 40.447666,-108.513720: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:41.985833
Currently retrieving data for 40.447666,-108.513720: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:43.482919
Currently retrieving data for 40.447666,-108.513720: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:45.130013
Currently retrieving data for 40.447666,-108.513720: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:46.970118
Currently retrieving data for 40.447666,-108.513720: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:48.323195
Currently retrieving data for 40.447666,-108.513720: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:49.694274
Currently retrieving data for 40.447666,-108.513720: from 2017-04-01 to 2017-04-30


In [33]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["24.627054,-82.872931"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 24.627054,-82.872931


Currently retrieving data for 24.627054,-82.872931: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:03.218184
Currently retrieving data for 24.627054,-82.872931: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.619264
Currently retrieving data for 24.627054,-82.872931: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.990342
Currently retrieving data for 24.627054,-82.872931: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.312418
Currently retrieving data for 24.627054,-82.872931: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.607492
Currently retrieving data for 24.627054,-82.872931: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.954569
Currently retrieving data for 24.627054,-82.872931: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.424653
Currently retrieving data for 24.627054,-82.872931: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:31.419228
Currently retrieving data for 24.627054,-82.872931: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.845310
Currently retrieving data for 24.627054,-82.872931: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.136384
Currently retrieving data for 24.627054,-82.872931: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.401456
Currently retrieving data for 24.627054,-82.872931: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.883541
Currently retrieving data for 24.627054,-82.872931: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.636641
Currently retrieving data for 24.627054,-82.872931: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.164729
Currently retrieving data for 24.627054,-82.872931: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.322852
Currently retrieving data for 24.627054,-82.872931: from 2017-04-01 to 2017-04-30
Time ela

In [34]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["25.399325,-80.655772"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 25.399325,-80.655772


Currently retrieving data for 25.399325,-80.655772: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.822105
Currently retrieving data for 25.399325,-80.655772: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.755215
Currently retrieving data for 25.399325,-80.655772: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.554318
Currently retrieving data for 25.399325,-80.655772: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.390423
Currently retrieving data for 25.399325,-80.655772: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:09.310533
Currently retrieving data for 25.399325,-80.655772: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:13.582777
Currently retrieving data for 25.399325,-80.655772: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:15.288875
Currently retrieving data for 25.399325,-80.655772: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:02:04.106438
Currently retrieving data for 25.399325,-80.655772: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:02:05.687524
Currently retrieving data for 25.399325,-80.655772: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:02:07.185605
Currently retrieving data for 25.399325,-80.655772: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:02:08.738659
Currently retrieving data for 25.399325,-80.655772: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:02:10.332699
Currently retrieving data for 25.399325,-80.655772: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:02:12.254809
Currently retrieving data for 25.399325,-80.655772: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:13.539882
Currently retrieving data for 25.399325,-80.655772: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:14.819955
Currently retrieving data for 25.399325,-80.655772: from 2017-04-01 to 2017-04-30
Time ela

In [35]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["38.214092,-80.888697"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 38.214092,-80.888697


Currently retrieving data for 38.214092,-80.888697: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.717464
Currently retrieving data for 38.214092,-80.888697: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.285735
Currently retrieving data for 38.214092,-80.888697: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.626411
Currently retrieving data for 38.214092,-80.888697: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.884681
Currently retrieving data for 38.214092,-80.888697: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.283761
Currently retrieving data for 38.214092,-80.888697: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.573835
Currently retrieving data for 38.214092,-80.888697: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.904911
Currently retrieving data for 38.214092,-80.888697: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:38.129957
Currently retrieving data for 38.214092,-80.888697: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:39.435032
Currently retrieving data for 38.214092,-80.888697: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:40.775109
Currently retrieving data for 38.214092,-80.888697: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:42.080183
Currently retrieving data for 38.214092,-80.888697: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:43.432261
Currently retrieving data for 38.214092,-80.888697: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:45.573383
Currently retrieving data for 38.214092,-80.888697: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:46.988464
Currently retrieving data for 38.214092,-80.888697: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:48.396544
Currently retrieving data for 38.214092,-80.888697: from 2017-04-01 to 2017-04-30
Time ela

In [36]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["40.451291,-73.991239"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 40.451291,-73.991239


Currently retrieving data for 40.451291,-73.991239: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.833104
Currently retrieving data for 40.451291,-73.991239: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.820218
Currently retrieving data for 40.451291,-73.991239: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.046345
Currently retrieving data for 40.451291,-73.991239: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.747443
Currently retrieving data for 40.451291,-73.991239: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:09.598549
Currently retrieving data for 40.451291,-73.991239: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:11.505658
Currently retrieving data for 40.451291,-73.991239: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:13.190491
Currently retrieving data for 40.451291,-73.991239: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:02:10.105169
Currently retrieving data for 40.451291,-73.991239: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:02:13.388357
Currently retrieving data for 40.451291,-73.991239: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:02:15.826497
Currently retrieving data for 40.451291,-73.991239: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:02:18.629657
Currently retrieving data for 40.451291,-73.991239: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:02:20.318754
Currently retrieving data for 40.451291,-73.991239: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:02:22.783895
Currently retrieving data for 40.451291,-73.991239: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:24.057967
Currently retrieving data for 40.451291,-73.991239: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:25.742064
Currently retrieving data for 40.451291,-73.991239: from 2017-04-01 to 2017-04-30
Time ela

In [37]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["39.821005,-77.246171"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 39.821005,-77.246171


Currently retrieving data for 39.821005,-77.246171: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.714098
Currently retrieving data for 39.821005,-77.246171: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.367192
Currently retrieving data for 39.821005,-77.246171: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.945283
Currently retrieving data for 39.821005,-77.246171: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.215355
Currently retrieving data for 39.821005,-77.246171: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.285474
Currently retrieving data for 39.821005,-77.246171: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.917166
Currently retrieving data for 39.821005,-77.246171: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:12.054482
Currently retrieving data for 39.821005,-77.246171: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:49.238265
Currently retrieving data for 39.821005,-77.246171: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:50.604343
Currently retrieving data for 39.821005,-77.246171: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:51.928418
Currently retrieving data for 39.821005,-77.246171: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:53.218492
Currently retrieving data for 39.821005,-77.246171: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:54.888588
Currently retrieving data for 39.821005,-77.246171: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:57.493737
Currently retrieving data for 39.821005,-77.246171: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:58.906818
Currently retrieving data for 39.821005,-77.246171: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:00.301897
Currently retrieving data for 39.821005,-77.246171: from 2017-04-01 to 2017-04-30
Time ela

In [38]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["48.957820,-114.318387"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 48.957820,-114.318387


Currently retrieving data for 48.957820,-114.318387: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.778102
Currently retrieving data for 48.957820,-114.318387: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.280245
Currently retrieving data for 48.957820,-114.318387: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.092348
Currently retrieving data for 48.957820,-114.318387: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:08.699498
Currently retrieving data for 48.957820,-114.318387: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:11.249643
Currently retrieving data for 48.957820,-114.318387: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:13.122751
Currently retrieving data for 48.957820,-114.318387: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:15.093863
Currently retrieving data for 48.957820,-114.318387: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:02:03.715382
Currently retrieving data for 48.957820,-114.318387: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:02:05.247469
Currently retrieving data for 48.957820,-114.318387: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:02:06.845561
Currently retrieving data for 48.957820,-114.318387: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:02:08.282643
Currently retrieving data for 48.957820,-114.318387: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:02:10.105747
Currently retrieving data for 48.957820,-114.318387: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:02:12.892907
Currently retrieving data for 48.957820,-114.318387: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:14.169980
Currently retrieving data for 48.957820,-114.318387: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:15.861077
Currently retrieving data for 48.957820,-114.318387: from 2017-04-01 to 2017-04-30


In [39]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.498985,-110.699117"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.498985,-110.699117


Currently retrieving data for 37.498985,-110.699117: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.907109
Currently retrieving data for 37.498985,-110.699117: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.805218
Currently retrieving data for 37.498985,-110.699117: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.453312
Currently retrieving data for 37.498985,-110.699117: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.582434
Currently retrieving data for 37.498985,-110.699117: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:09.431540
Currently retrieving data for 37.498985,-110.699117: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:12.294703
Currently retrieving data for 37.498985,-110.699117: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:13.501773
Currently retrieving data for 37.498985,-110.699117: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:46.127070
Currently retrieving data for 37.498985,-110.699117: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:47.422144
Currently retrieving data for 37.498985,-110.699117: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:49.059238
Currently retrieving data for 37.498985,-110.699117: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:50.309310
Currently retrieving data for 37.498985,-110.699117: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:52.592440
Currently retrieving data for 37.498985,-110.699117: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:54.417545
Currently retrieving data for 37.498985,-110.699117: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:55.798624
Currently retrieving data for 37.498985,-110.699117: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:57.447718
Currently retrieving data for 37.498985,-110.699117: from 2017-04-01 to 2017-04-30


In [40]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["38.827294,-127.035260"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 38.827294,-127.035260


Currently retrieving data for 38.827294,-127.035260: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.843105
Currently retrieving data for 38.827294,-127.035260: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.748271
Currently retrieving data for 38.827294,-127.035260: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.432368
Currently retrieving data for 38.827294,-127.035260: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:08.170467
Currently retrieving data for 38.827294,-127.035260: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:12.501715
Currently retrieving data for 38.827294,-127.035260: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:14.392823
Currently retrieving data for 38.827294,-127.035260: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:16.004915
Currently retrieving data for 38.827294,-127.035260: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:57.802323
Currently retrieving data for 38.827294,-127.035260: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:59.354412
Currently retrieving data for 38.827294,-127.035260: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:02:01.035508
Currently retrieving data for 38.827294,-127.035260: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:02:02.611598
Currently retrieving data for 38.827294,-127.035260: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:02:04.117685
Currently retrieving data for 38.827294,-127.035260: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:02:06.081797
Currently retrieving data for 38.827294,-127.035260: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:07.348869
Currently retrieving data for 38.827294,-127.035260: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:08.639943
Currently retrieving data for 38.827294,-127.035260: from 2017-04-01 to 2017-04-30


In [41]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["36.050587,-112.121138"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 36.050587,-112.121138


Currently retrieving data for 36.050587,-112.121138: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.529087
Currently retrieving data for 36.050587,-112.121138: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.003171
Currently retrieving data for 36.050587,-112.121138: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.372250
Currently retrieving data for 36.050587,-112.121138: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.680325
Currently retrieving data for 36.050587,-112.121138: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.581433
Currently retrieving data for 36.050587,-112.121138: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.877507
Currently retrieving data for 36.050587,-112.121138: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.680611
Currently retrieving data for 36.050587,-112.121138: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:31.646242
Currently retrieving data for 36.050587,-112.121138: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.897313
Currently retrieving data for 36.050587,-112.121138: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.229389
Currently retrieving data for 36.050587,-112.121138: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.565466
Currently retrieving data for 36.050587,-112.121138: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.905542
Currently retrieving data for 36.050587,-112.121138: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.987661
Currently retrieving data for 36.050587,-112.121138: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.364740
Currently retrieving data for 36.050587,-112.121138: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.634813
Currently retrieving data for 36.050587,-112.121138: from 2017-04-01 to 2017-04-30


In [ ]:
a

In [42]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["38.979080,-76.898410"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 38.979080,-76.898410


Currently retrieving data for 38.979080,-76.898410: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.797103
Currently retrieving data for 38.979080,-76.898410: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.439197
Currently retrieving data for 38.979080,-76.898410: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.129294
Currently retrieving data for 38.979080,-76.898410: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.885394
Currently retrieving data for 38.979080,-76.898410: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.493486
Currently retrieving data for 38.979080,-76.898410: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:10.253587
Currently retrieving data for 38.979080,-76.898410: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.850678
Currently retrieving data for 38.979080,-76.898410: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:49.309252
Currently retrieving data for 38.979080,-76.898410: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:50.701332
Currently retrieving data for 38.979080,-76.898410: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:52.291423
Currently retrieving data for 38.979080,-76.898410: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:53.772508
Currently retrieving data for 38.979080,-76.898410: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:55.339597
Currently retrieving data for 38.979080,-76.898410: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:57.622728
Currently retrieving data for 38.979080,-76.898410: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:59.043809
Currently retrieving data for 38.979080,-76.898410: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:00.386886
Currently retrieving data for 38.979080,-76.898410: from 2017-04-01 to 2017-04-30
Time ela

In [43]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["48.003609,-89.828387"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 48.003609,-89.828387


Currently retrieving data for 48.003609,-89.828387: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.382079
Currently retrieving data for 48.003609,-89.828387: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.845163
Currently retrieving data for 48.003609,-89.828387: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.201241
Currently retrieving data for 48.003609,-89.828387: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.494315
Currently retrieving data for 48.003609,-89.828387: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.721385
Currently retrieving data for 48.003609,-89.828387: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.060461
Currently retrieving data for 48.003609,-89.828387: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.476542
Currently retrieving data for 48.003609,-89.828387: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:32.900314
Currently retrieving data for 48.003609,-89.828387: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:34.131384
Currently retrieving data for 48.003609,-89.828387: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:35.389456
Currently retrieving data for 48.003609,-89.828387: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.609526
Currently retrieving data for 48.003609,-89.828387: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.860598
Currently retrieving data for 48.003609,-89.828387: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.566695
Currently retrieving data for 48.003609,-89.828387: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.803766
Currently retrieving data for 48.003609,-89.828387: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.061838
Currently retrieving data for 48.003609,-89.828387: from 2017-04-01 to 2017-04-30
Time ela

In [44]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["35.659676,-83.586382"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 35.659676,-83.586382


Currently retrieving data for 35.659676,-83.586382: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.427082
Currently retrieving data for 35.659676,-83.586382: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.878165
Currently retrieving data for 35.659676,-83.586382: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.186240
Currently retrieving data for 35.659676,-83.586382: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.443312
Currently retrieving data for 35.659676,-83.586382: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.759387
Currently retrieving data for 35.659676,-83.586382: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.186469
Currently retrieving data for 35.659676,-83.586382: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.451541
Currently retrieving data for 35.659676,-83.586382: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:31.846254
Currently retrieving data for 35.659676,-83.586382: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:33.148328
Currently retrieving data for 35.659676,-83.586382: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.492405
Currently retrieving data for 35.659676,-83.586382: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.824481
Currently retrieving data for 35.659676,-83.586382: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.477576
Currently retrieving data for 35.659676,-83.586382: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.543694
Currently retrieving data for 35.659676,-83.586382: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.938774
Currently retrieving data for 35.659676,-83.586382: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.400857
Currently retrieving data for 35.659676,-83.586382: from 2017-04-01 to 2017-04-30
Time ela

In [45]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["43.616323,-110.666872"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 43.616323,-110.666872


Currently retrieving data for 43.616323,-110.666872: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.510086
Currently retrieving data for 43.616323,-110.666872: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.940168
Currently retrieving data for 43.616323,-110.666872: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:10.296589
Currently retrieving data for 43.616323,-110.666872: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:11.485657
Currently retrieving data for 43.616323,-110.666872: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:12.720727
Currently retrieving data for 43.616323,-110.666872: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:14.254815
Currently retrieving data for 43.616323,-110.666872: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:15.569890
Currently retrieving data for 43.616323,-110.666872: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:36.986546
Currently retrieving data for 43.616323,-110.666872: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:38.286620
Currently retrieving data for 43.616323,-110.666872: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:39.591695
Currently retrieving data for 43.616323,-110.666872: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:40.787763
Currently retrieving data for 43.616323,-110.666872: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:41.967831
Currently retrieving data for 43.616323,-110.666872: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:44.574980
Currently retrieving data for 43.616323,-110.666872: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:45.826052
Currently retrieving data for 43.616323,-110.666872: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:47.131126
Currently retrieving data for 43.616323,-110.666872: from 2017-04-01 to 2017-04-30


In [46]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["30.397808,-88.795595"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 30.397808,-88.795595


Currently retrieving data for 30.397808,-88.795595: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.416081
Currently retrieving data for 30.397808,-88.795595: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.980170
Currently retrieving data for 30.397808,-88.795595: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.322362
Currently retrieving data for 30.397808,-88.795595: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.607435
Currently retrieving data for 30.397808,-88.795595: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.879508
Currently retrieving data for 30.397808,-88.795595: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:10.155581
Currently retrieving data for 30.397808,-88.795595: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.456655
Currently retrieving data for 30.397808,-88.795595: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:31.669618
Currently retrieving data for 30.397808,-88.795595: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.907689
Currently retrieving data for 30.397808,-88.795595: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.156760
Currently retrieving data for 30.397808,-88.795595: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.403832
Currently retrieving data for 30.397808,-88.795595: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.780910
Currently retrieving data for 30.397808,-88.795595: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.505009
Currently retrieving data for 30.397808,-88.795595: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.802140
Currently retrieving data for 30.397808,-88.795595: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.342229
Currently retrieving data for 30.397808,-88.795595: from 2017-04-01 to 2017-04-30
Time ela

In [47]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["31.895262,-104.826341"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 31.895262,-104.826341


Currently retrieving data for 31.895262,-104.826341: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.736100
Currently retrieving data for 31.895262,-104.826341: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.073176
Currently retrieving data for 31.895262,-104.826341: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.387251
Currently retrieving data for 31.895262,-104.826341: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:12.670725
Currently retrieving data for 31.895262,-104.826341: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:13.972799
Currently retrieving data for 31.895262,-104.826341: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:15.296875
Currently retrieving data for 31.895262,-104.826341: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:17.029974
Currently retrieving data for 31.895262,-104.826341: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:48.082182
Currently retrieving data for 31.895262,-104.826341: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:49.371256
Currently retrieving data for 31.895262,-104.826341: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:50.726333
Currently retrieving data for 31.895262,-104.826341: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:52.071410
Currently retrieving data for 31.895262,-104.826341: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:53.424488
Currently retrieving data for 31.895262,-104.826341: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:55.224591
Currently retrieving data for 31.895262,-104.826341: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:56.670673
Currently retrieving data for 31.895262,-104.826341: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:58.055753
Currently retrieving data for 31.895262,-104.826341: from 2017-04-01 to 2017-04-30


In [48]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["20.667755,-156.003694"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 20.667755,-156.003694


Currently retrieving data for 20.667755,-156.003694: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.517086
Currently retrieving data for 20.667755,-156.003694: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.008172
Currently retrieving data for 20.667755,-156.003694: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.492257
Currently retrieving data for 20.667755,-156.003694: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.948340
Currently retrieving data for 20.667755,-156.003694: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.406423
Currently retrieving data for 20.667755,-156.003694: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.872507
Currently retrieving data for 20.667755,-156.003694: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.532602
Currently retrieving data for 20.667755,-156.003694: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:46.295079
Currently retrieving data for 20.667755,-156.003694: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:47.678158
Currently retrieving data for 20.667755,-156.003694: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:49.099240
Currently retrieving data for 20.667755,-156.003694: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:50.665329
Currently retrieving data for 20.667755,-156.003694: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:52.108412
Currently retrieving data for 20.667755,-156.003694: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:53.956518
Currently retrieving data for 20.667755,-156.003694: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:55.430602
Currently retrieving data for 20.667755,-156.003694: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:56.753678
Currently retrieving data for 20.667755,-156.003694: from 2017-04-01 to 2017-04-30


In [49]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["19.343150,-155.273610"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 19.343150,-155.273610


Currently retrieving data for 19.343150,-155.273610: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.797102
Currently retrieving data for 19.343150,-155.273610: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.003229
Currently retrieving data for 19.343150,-155.273610: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.631322
Currently retrieving data for 19.343150,-155.273610: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.407423
Currently retrieving data for 19.343150,-155.273610: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.755500
Currently retrieving data for 19.343150,-155.273610: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:10.081576
Currently retrieving data for 19.343150,-155.273610: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.358649
Currently retrieving data for 19.343150,-155.273610: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:33.979375
Currently retrieving data for 19.343150,-155.273610: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.241447
Currently retrieving data for 19.343150,-155.273610: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:36.591524
Currently retrieving data for 19.343150,-155.273610: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:37.848795
Currently retrieving data for 19.343150,-155.273610: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:39.124868
Currently retrieving data for 19.343150,-155.273610: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:40.913970
Currently retrieving data for 19.343150,-155.273610: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.317050
Currently retrieving data for 19.343150,-155.273610: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:43.848138
Currently retrieving data for 19.343150,-155.273610: from 2017-04-01 to 2017-04-30


In [50]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["41.671966,-86.983786"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 41.671966,-86.983786


Currently retrieving data for 41.671966,-86.983786: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.687096
Currently retrieving data for 41.671966,-86.983786: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.306189
Currently retrieving data for 41.671966,-86.983786: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.775273
Currently retrieving data for 41.671966,-86.983786: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.222356
Currently retrieving data for 41.671966,-86.983786: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.502429
Currently retrieving data for 41.671966,-86.983786: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.765501
Currently retrieving data for 41.671966,-86.983786: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.077576
Currently retrieving data for 41.671966,-86.983786: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:34.605367
Currently retrieving data for 41.671966,-86.983786: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.904441
Currently retrieving data for 41.671966,-86.983786: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:37.279520
Currently retrieving data for 41.671966,-86.983786: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:39.046621
Currently retrieving data for 41.671966,-86.983786: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:40.325694
Currently retrieving data for 41.671966,-86.983786: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:43.544878
Currently retrieving data for 41.671966,-86.983786: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:44.843953
Currently retrieving data for 41.671966,-86.983786: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:46.172029
Currently retrieving data for 41.671966,-86.983786: from 2017-04-01 to 2017-04-30
Time ela

In [51]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["47.904435,-89.174178"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 47.904435,-89.174178


Currently retrieving data for 47.904435,-89.174178: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.407081
Currently retrieving data for 47.904435,-89.174178: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.822162
Currently retrieving data for 47.904435,-89.174178: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.124236
Currently retrieving data for 47.904435,-89.174178: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.432311
Currently retrieving data for 47.904435,-89.174178: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.804389
Currently retrieving data for 47.904435,-89.174178: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.140466
Currently retrieving data for 47.904435,-89.174178: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.715556
Currently retrieving data for 47.904435,-89.174178: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:38.097176
Currently retrieving data for 47.904435,-89.174178: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:39.529258
Currently retrieving data for 47.904435,-89.174178: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:40.893336
Currently retrieving data for 47.904435,-89.174178: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:42.209553
Currently retrieving data for 47.904435,-89.174178: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:43.545560
Currently retrieving data for 47.904435,-89.174178: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:45.101222
Currently retrieving data for 47.904435,-89.174178: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:46.635844
Currently retrieving data for 47.904435,-89.174178: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:48.026652
Currently retrieving data for 47.904435,-89.174178: from 2017-04-01 to 2017-04-30
Time ela

In [52]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["34.002019,-116.019604"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 34.002019,-116.019604


Currently retrieving data for 34.002019,-116.019604: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.610092
Currently retrieving data for 34.002019,-116.019604: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.266187
Currently retrieving data for 34.002019,-116.019604: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.682268
Currently retrieving data for 34.002019,-116.019604: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.045346
Currently retrieving data for 34.002019,-116.019604: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.407424
Currently retrieving data for 34.002019,-116.019604: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.002515
Currently retrieving data for 34.002019,-116.019604: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.326590
Currently retrieving data for 34.002019,-116.019604: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:35.580467
Currently retrieving data for 34.002019,-116.019604: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:36.826538
Currently retrieving data for 34.002019,-116.019604: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:40.910772
Currently retrieving data for 34.002019,-116.019604: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:42.218846
Currently retrieving data for 34.002019,-116.019604: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:43.618926
Currently retrieving data for 34.002019,-116.019604: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:45.680044
Currently retrieving data for 34.002019,-116.019604: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:46.984119
Currently retrieving data for 34.002019,-116.019604: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:49.289251
Currently retrieving data for 34.002019,-116.019604: from 2017-04-01 to 2017-04-30


In [53]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["59.575423,-135.334877"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 59.575423,-135.334877


Currently retrieving data for 59.575423,-135.334877: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.159067
Currently retrieving data for 59.575423,-135.334877: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.720156
Currently retrieving data for 59.575423,-135.334877: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.016230
Currently retrieving data for 59.575423,-135.334877: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.294303
Currently retrieving data for 59.575423,-135.334877: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.599378
Currently retrieving data for 59.575423,-135.334877: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.034460
Currently retrieving data for 59.575423,-135.334877: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.442540
Currently retrieving data for 59.575423,-135.334877: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:35.281450
Currently retrieving data for 59.575423,-135.334877: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:36.628527
Currently retrieving data for 59.575423,-135.334877: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:38.026607
Currently retrieving data for 59.575423,-135.334877: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:39.362684
Currently retrieving data for 59.575423,-135.334877: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:40.698760
Currently retrieving data for 59.575423,-135.334877: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:42.283851
Currently retrieving data for 59.575423,-135.334877: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:43.956946
Currently retrieving data for 59.575423,-135.334877: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:45.454032
Currently retrieving data for 59.575423,-135.334877: from 2017-04-01 to 2017-04-30


In [54]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["41.717722,-121.504111"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 41.717722,-121.504111


Currently retrieving data for 41.717722,-121.504111: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.535087
Currently retrieving data for 41.717722,-121.504111: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.395194
Currently retrieving data for 41.717722,-121.504111: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.095291
Currently retrieving data for 41.717722,-121.504111: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.727384
Currently retrieving data for 41.717722,-121.504111: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.160466
Currently retrieving data for 41.717722,-121.504111: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.575547
Currently retrieving data for 41.717722,-121.504111: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.935625
Currently retrieving data for 41.717722,-121.504111: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:39.378684
Currently retrieving data for 41.717722,-121.504111: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:40.676758
Currently retrieving data for 41.717722,-121.504111: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:42.224846
Currently retrieving data for 41.717722,-121.504111: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:43.507920
Currently retrieving data for 41.717722,-121.504111: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:44.779993
Currently retrieving data for 41.717722,-121.504111: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:47.183130
Currently retrieving data for 41.717722,-121.504111: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:48.734219
Currently retrieving data for 41.717722,-121.504111: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:50.118298
Currently retrieving data for 41.717722,-121.504111: from 2017-04-01 to 2017-04-30


In [55]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["35.223622,-114.559874"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 35.223622,-114.559874


Currently retrieving data for 35.223622,-114.559874: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.290131
Currently retrieving data for 35.223622,-114.559874: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.212241
Currently retrieving data for 35.223622,-114.559874: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.680325
Currently retrieving data for 35.223622,-114.559874: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.022402
Currently retrieving data for 35.223622,-114.559874: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.487485
Currently retrieving data for 35.223622,-114.559874: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.887566
Currently retrieving data for 35.223622,-114.559874: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.207641
Currently retrieving data for 35.223622,-114.559874: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:34.564409
Currently retrieving data for 35.223622,-114.559874: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:36.230504
Currently retrieving data for 35.223622,-114.559874: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:37.681587
Currently retrieving data for 35.223622,-114.559874: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:39.112669
Currently retrieving data for 35.223622,-114.559874: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:40.489748
Currently retrieving data for 35.223622,-114.559874: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:42.625870
Currently retrieving data for 35.223622,-114.559874: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:43.999948
Currently retrieving data for 35.223622,-114.559874: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:45.343025
Currently retrieving data for 35.223622,-114.559874: from 2017-04-01 to 2017-04-30


In [56]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["35.650267,-101.621614"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 35.650267,-101.621614


Currently retrieving data for 35.650267,-101.621614: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.613092
Currently retrieving data for 35.650267,-101.621614: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.209241
Currently retrieving data for 35.650267,-101.621614: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.650323
Currently retrieving data for 35.650267,-101.621614: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.879394
Currently retrieving data for 35.650267,-101.621614: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.207470
Currently retrieving data for 35.650267,-101.621614: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.579548
Currently retrieving data for 35.650267,-101.621614: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.086634
Currently retrieving data for 35.650267,-101.621614: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:34.808423
Currently retrieving data for 35.650267,-101.621614: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:36.181501
Currently retrieving data for 35.650267,-101.621614: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:37.520578
Currently retrieving data for 35.650267,-101.621614: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:38.947660
Currently retrieving data for 35.650267,-101.621614: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:40.292737
Currently retrieving data for 35.650267,-101.621614: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:42.433859
Currently retrieving data for 35.650267,-101.621614: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:44.099954
Currently retrieving data for 35.650267,-101.621614: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:45.561038
Currently retrieving data for 35.650267,-101.621614: from 2017-04-01 to 2017-04-30


In [57]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["48.601125,-118.122312"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 48.601125,-118.122312


Currently retrieving data for 48.601125,-118.122312: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:03.146180
Currently retrieving data for 48.601125,-118.122312: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.631265
Currently retrieving data for 48.601125,-118.122312: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.033345
Currently retrieving data for 48.601125,-118.122312: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.485428
Currently retrieving data for 48.601125,-118.122312: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.898509
Currently retrieving data for 48.601125,-118.122312: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:10.562604
Currently retrieving data for 48.601125,-118.122312: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.903681
Currently retrieving data for 48.601125,-118.122312: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:36.369512
Currently retrieving data for 48.601125,-118.122312: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:37.685588
Currently retrieving data for 48.601125,-118.122312: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:39.074667
Currently retrieving data for 48.601125,-118.122312: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:40.424744
Currently retrieving data for 48.601125,-118.122312: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:41.837825
Currently retrieving data for 48.601125,-118.122312: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:43.764935
Currently retrieving data for 48.601125,-118.122312: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:45.084011
Currently retrieving data for 48.601125,-118.122312: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:46.432088
Currently retrieving data for 48.601125,-118.122312: from 2017-04-01 to 2017-04-30


In [58]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["40.564883,-121.305605"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 40.564883,-121.305605


Currently retrieving data for 40.564883,-121.305605: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.196125
Currently retrieving data for 40.564883,-121.305605: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.663209
Currently retrieving data for 40.564883,-121.305605: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:07.037402
Currently retrieving data for 40.564883,-121.305605: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:08.320475
Currently retrieving data for 40.564883,-121.305605: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:09.611549
Currently retrieving data for 40.564883,-121.305605: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:10.886622
Currently retrieving data for 40.564883,-121.305605: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:12.158695
Currently retrieving data for 40.564883,-121.305605: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:40.447745
Currently retrieving data for 40.564883,-121.305605: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:41.767820
Currently retrieving data for 40.564883,-121.305605: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:43.069895
Currently retrieving data for 40.564883,-121.305605: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:44.307966
Currently retrieving data for 40.564883,-121.305605: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:45.594039
Currently retrieving data for 40.564883,-121.305605: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:47.508149
Currently retrieving data for 40.564883,-121.305605: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:48.887228
Currently retrieving data for 40.564883,-121.305605: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:50.318309
Currently retrieving data for 40.564883,-121.305605: from 2017-04-01 to 2017-04-30


In [59]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.182189,-86.097043"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.182189,-86.097043


Currently retrieving data for 37.182189,-86.097043: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.630093
Currently retrieving data for 37.182189,-86.097043: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.156180
Currently retrieving data for 37.182189,-86.097043: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.498257
Currently retrieving data for 37.182189,-86.097043: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.801331
Currently retrieving data for 37.182189,-86.097043: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.341419
Currently retrieving data for 37.182189,-86.097043: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.807503
Currently retrieving data for 37.182189,-86.097043: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.101577
Currently retrieving data for 37.182189,-86.097043: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:35.192638
Currently retrieving data for 37.182189,-86.097043: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:36.612719
Currently retrieving data for 37.182189,-86.097043: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:38.434824
Currently retrieving data for 37.182189,-86.097043: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:39.971912
Currently retrieving data for 37.182189,-86.097043: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:41.431995
Currently retrieving data for 37.182189,-86.097043: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:43.540116
Currently retrieving data for 37.182189,-86.097043: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:45.059203
Currently retrieving data for 37.182189,-86.097043: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:46.437281
Currently retrieving data for 37.182189,-86.097043: from 2017-04-01 to 2017-04-30
Time ela

In [60]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["34.943387,-115.512249"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 34.943387,-115.512249


Currently retrieving data for 34.943387,-115.512249: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.729156
Currently retrieving data for 34.943387,-115.512249: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.309246
Currently retrieving data for 34.943387,-115.512249: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.766330
Currently retrieving data for 34.943387,-115.512249: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.046403
Currently retrieving data for 34.943387,-115.512249: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.309475
Currently retrieving data for 34.943387,-115.512249: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.586548
Currently retrieving data for 34.943387,-115.512249: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.856621
Currently retrieving data for 34.943387,-115.512249: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:38.528635
Currently retrieving data for 34.943387,-115.512249: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:39.838710
Currently retrieving data for 34.943387,-115.512249: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:41.110783
Currently retrieving data for 34.943387,-115.512249: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:42.358854
Currently retrieving data for 34.943387,-115.512249: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:43.680930
Currently retrieving data for 34.943387,-115.512249: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:46.057066
Currently retrieving data for 34.943387,-115.512249: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:47.412143
Currently retrieving data for 34.943387,-115.512249: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:48.773221
Currently retrieving data for 34.943387,-115.512249: from 2017-04-01 to 2017-04-30


In [61]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["46.767688,-121.792210"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 46.767688,-121.792210


Currently retrieving data for 46.767688,-121.792210: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.765101
Currently retrieving data for 46.767688,-121.792210: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.448197
Currently retrieving data for 46.767688,-121.792210: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.892279
Currently retrieving data for 46.767688,-121.792210: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.172353
Currently retrieving data for 46.767688,-121.792210: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.518430
Currently retrieving data for 46.767688,-121.792210: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.838505
Currently retrieving data for 46.767688,-121.792210: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.113578
Currently retrieving data for 46.767688,-121.792210: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:33.779364
Currently retrieving data for 46.767688,-121.792210: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.076438
Currently retrieving data for 46.767688,-121.792210: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:36.400513
Currently retrieving data for 46.767688,-121.792210: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:37.685587
Currently retrieving data for 46.767688,-121.792210: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:39.309680
Currently retrieving data for 46.767688,-121.792210: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:41.377798
Currently retrieving data for 46.767688,-121.792210: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.958889
Currently retrieving data for 46.767688,-121.792210: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:44.311966
Currently retrieving data for 46.767688,-121.792210: from 2017-04-01 to 2017-04-30


In [62]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.609218,-109.984193"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.609218,-109.984193


Currently retrieving data for 37.609218,-109.984193: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.436082
Currently retrieving data for 37.609218,-109.984193: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.065175
Currently retrieving data for 37.609218,-109.984193: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.475256
Currently retrieving data for 37.609218,-109.984193: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.752329
Currently retrieving data for 37.609218,-109.984193: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.019401
Currently retrieving data for 37.609218,-109.984193: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.355478
Currently retrieving data for 37.609218,-109.984193: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.600549
Currently retrieving data for 37.609218,-109.984193: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:31.901199
Currently retrieving data for 37.609218,-109.984193: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:33.189273
Currently retrieving data for 37.609218,-109.984193: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.503348
Currently retrieving data for 37.609218,-109.984193: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.819423
Currently retrieving data for 37.609218,-109.984193: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.159500
Currently retrieving data for 37.609218,-109.984193: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.410629
Currently retrieving data for 37.609218,-109.984193: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.945717
Currently retrieving data for 37.609218,-109.984193: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.435802
Currently retrieving data for 37.609218,-109.984193: from 2017-04-01 to 2017-04-30


In [63]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["32.087431,-90.798840"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 32.087431,-90.798840


Currently retrieving data for 32.087431,-90.798840: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.643094
Currently retrieving data for 32.087431,-90.798840: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.280188
Currently retrieving data for 32.087431,-90.798840: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.070290
Currently retrieving data for 32.087431,-90.798840: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.767387
Currently retrieving data for 32.087431,-90.798840: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.545489
Currently retrieving data for 32.087431,-90.798840: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:10.250586
Currently retrieving data for 32.087431,-90.798840: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:12.064690
Currently retrieving data for 32.087431,-90.798840: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:49.329629
Currently retrieving data for 32.087431,-90.798840: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:50.791713
Currently retrieving data for 32.087431,-90.798840: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:52.222795
Currently retrieving data for 32.087431,-90.798840: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:53.572872
Currently retrieving data for 32.087431,-90.798840: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:55.097959
Currently retrieving data for 32.087431,-90.798840: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:57.068072
Currently retrieving data for 32.087431,-90.798840: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:58.562157
Currently retrieving data for 32.087431,-90.798840: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:59.983239
Currently retrieving data for 32.087431,-90.798840: from 2017-04-01 to 2017-04-30
Time ela

In [64]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["36.787319,-118.665095"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 36.787319,-118.665095


Currently retrieving data for 36.787319,-118.665095: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.454083
Currently retrieving data for 36.787319,-118.665095: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.825162
Currently retrieving data for 36.787319,-118.665095: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.216241
Currently retrieving data for 36.787319,-118.665095: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.626322
Currently retrieving data for 36.787319,-118.665095: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.960398
Currently retrieving data for 36.787319,-118.665095: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.252472
Currently retrieving data for 36.787319,-118.665095: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.535546
Currently retrieving data for 36.787319,-118.665095: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:32.427287
Currently retrieving data for 36.787319,-118.665095: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:33.737362
Currently retrieving data for 36.787319,-118.665095: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.992433
Currently retrieving data for 36.787319,-118.665095: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.347511
Currently retrieving data for 36.787319,-118.665095: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.687588
Currently retrieving data for 36.787319,-118.665095: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:40.059723
Currently retrieving data for 36.787319,-118.665095: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:41.556809
Currently retrieving data for 36.787319,-118.665095: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.924887
Currently retrieving data for 36.787319,-118.665095: from 2017-04-01 to 2017-04-30


In [65]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.829446,-81.013398"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.829446,-81.013398


Currently retrieving data for 37.829446,-81.013398: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.541088
Currently retrieving data for 37.829446,-81.013398: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.126179
Currently retrieving data for 37.829446,-81.013398: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.433253
Currently retrieving data for 37.829446,-81.013398: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.840334
Currently retrieving data for 37.829446,-81.013398: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.136408
Currently retrieving data for 37.829446,-81.013398: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.426482
Currently retrieving data for 37.829446,-81.013398: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.737557
Currently retrieving data for 37.829446,-81.013398: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:32.273596
Currently retrieving data for 37.829446,-81.013398: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:33.594672
Currently retrieving data for 37.829446,-81.013398: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:35.174762
Currently retrieving data for 37.829446,-81.013398: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.465836
Currently retrieving data for 37.829446,-81.013398: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.864916
Currently retrieving data for 37.829446,-81.013398: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.944035
Currently retrieving data for 37.829446,-81.013398: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:41.290112
Currently retrieving data for 37.829446,-81.013398: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.837200
Currently retrieving data for 37.829446,-81.013398: from 2017-04-01 to 2017-04-30
Time ela

In [66]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["48.671000,-121.261000"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 48.671000,-121.261000


Currently retrieving data for 48.671000,-121.261000: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.362078
Currently retrieving data for 48.671000,-121.261000: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.832162
Currently retrieving data for 48.671000,-121.261000: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.223242
Currently retrieving data for 48.671000,-121.261000: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.515316
Currently retrieving data for 48.671000,-121.261000: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.904395
Currently retrieving data for 48.671000,-121.261000: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.226471
Currently retrieving data for 48.671000,-121.261000: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.877565
Currently retrieving data for 48.671000,-121.261000: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:33.733361
Currently retrieving data for 48.671000,-121.261000: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.075438
Currently retrieving data for 48.671000,-121.261000: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:36.333510
Currently retrieving data for 48.671000,-121.261000: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:37.626584
Currently retrieving data for 48.671000,-121.261000: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:38.947660
Currently retrieving data for 48.671000,-121.261000: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:40.769764
Currently retrieving data for 48.671000,-121.261000: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.063838
Currently retrieving data for 48.671000,-121.261000: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:43.649929
Currently retrieving data for 48.671000,-121.261000: from 2017-04-01 to 2017-04-30


In [67]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["36.069637,-84.662876"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 36.069637,-84.662876


Currently retrieving data for 36.069637,-84.662876: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.450083
Currently retrieving data for 36.069637,-84.662876: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.040174
Currently retrieving data for 36.069637,-84.662876: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.475256
Currently retrieving data for 36.069637,-84.662876: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.873336
Currently retrieving data for 36.069637,-84.662876: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.452426
Currently retrieving data for 36.069637,-84.662876: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.824505
Currently retrieving data for 36.069637,-84.662876: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.048575
Currently retrieving data for 36.069637,-84.662876: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:31.575151
Currently retrieving data for 36.069637,-84.662876: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.811222
Currently retrieving data for 36.069637,-84.662876: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.082295
Currently retrieving data for 36.069637,-84.662876: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.496376
Currently retrieving data for 36.069637,-84.662876: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.753448
Currently retrieving data for 36.069637,-84.662876: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.790564
Currently retrieving data for 36.069637,-84.662876: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.460660
Currently retrieving data for 36.069637,-84.662876: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.843739
Currently retrieving data for 36.069637,-84.662876: from 2017-04-01 to 2017-04-30
Time ela

In [68]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["47.737303,-123.164701"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 47.737303,-123.164701


Currently retrieving data for 47.737303,-123.164701: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.400080
Currently retrieving data for 47.737303,-123.164701: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.022173
Currently retrieving data for 47.737303,-123.164701: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.663267
Currently retrieving data for 47.737303,-123.164701: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.244357
Currently retrieving data for 47.737303,-123.164701: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.704441
Currently retrieving data for 47.737303,-123.164701: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.297532
Currently retrieving data for 47.737303,-123.164701: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.822619
Currently retrieving data for 47.737303,-123.164701: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:48.500206
Currently retrieving data for 47.737303,-123.164701: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:49.837282
Currently retrieving data for 47.737303,-123.164701: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:51.376370
Currently retrieving data for 47.737303,-123.164701: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:52.699446
Currently retrieving data for 47.737303,-123.164701: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:54.239534
Currently retrieving data for 47.737303,-123.164701: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:56.140643
Currently retrieving data for 47.737303,-123.164701: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:57.538723
Currently retrieving data for 47.737303,-123.164701: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:58.930802
Currently retrieving data for 47.737303,-123.164701: from 2017-04-01 to 2017-04-30


In [69]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["34.013273,-111.723984"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 34.013273,-111.723984


Currently retrieving data for 34.013273,-111.723984: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.542088
Currently retrieving data for 34.013273,-111.723984: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.018172
Currently retrieving data for 34.013273,-111.723984: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.344248
Currently retrieving data for 34.013273,-111.723984: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.610320
Currently retrieving data for 34.013273,-111.723984: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.923396
Currently retrieving data for 34.013273,-111.723984: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.243471
Currently retrieving data for 34.013273,-111.723984: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.791560
Currently retrieving data for 34.013273,-111.723984: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:32.927315
Currently retrieving data for 34.013273,-111.723984: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:34.201388
Currently retrieving data for 34.013273,-111.723984: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:35.484461
Currently retrieving data for 34.013273,-111.723984: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.763534
Currently retrieving data for 34.013273,-111.723984: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:38.126612
Currently retrieving data for 34.013273,-111.723984: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:40.917772
Currently retrieving data for 34.013273,-111.723984: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.188844
Currently retrieving data for 34.013273,-111.723984: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:43.531921
Currently retrieving data for 34.013273,-111.723984: from 2017-04-01 to 2017-04-30


In [70]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["32.072190,-112.727959"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 32.072190,-112.727959


Currently retrieving data for 32.072190,-112.727959: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.541088
Currently retrieving data for 32.072190,-112.727959: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.987171
Currently retrieving data for 32.072190,-112.727959: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.323247
Currently retrieving data for 32.072190,-112.727959: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.525316
Currently retrieving data for 32.072190,-112.727959: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.861393
Currently retrieving data for 32.072190,-112.727959: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.122465
Currently retrieving data for 32.072190,-112.727959: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.440540
Currently retrieving data for 32.072190,-112.727959: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:31.540236
Currently retrieving data for 32.072190,-112.727959: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.738305
Currently retrieving data for 32.072190,-112.727959: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.765420
Currently retrieving data for 32.072190,-112.727959: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.381513
Currently retrieving data for 32.072190,-112.727959: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.663586
Currently retrieving data for 32.072190,-112.727959: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.721704
Currently retrieving data for 32.072190,-112.727959: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:41.150786
Currently retrieving data for 32.072190,-112.727959: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.422858
Currently retrieving data for 32.072190,-112.727959: from 2017-04-01 to 2017-04-30


In [72]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["36.964123,-90.982590"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 36.964123,-90.982590


Currently retrieving data for 36.964123,-90.982590: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.694097
Currently retrieving data for 36.964123,-90.982590: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.126179
Currently retrieving data for 36.964123,-90.982590: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.544260
Currently retrieving data for 36.964123,-90.982590: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.936339
Currently retrieving data for 36.964123,-90.982590: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.316418
Currently retrieving data for 36.964123,-90.982590: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.680496
Currently retrieving data for 36.964123,-90.982590: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.984571
Currently retrieving data for 36.964123,-90.982590: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:32.079065
Currently retrieving data for 36.964123,-90.982590: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:33.544119
Currently retrieving data for 36.964123,-90.982590: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.869160
Currently retrieving data for 36.964123,-90.982590: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.110166
Currently retrieving data for 36.964123,-90.982590: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.352022
Currently retrieving data for 36.964123,-90.982590: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.375840
Currently retrieving data for 36.964123,-90.982590: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.744500
Currently retrieving data for 36.964123,-90.982590: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.114738
Currently retrieving data for 36.964123,-90.982590: from 2017-04-01 to 2017-04-30
Time ela

In [73]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["27.483362,-97.309704"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 27.483362,-97.309704


Currently retrieving data for 27.483362,-97.309704: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.338134
Currently retrieving data for 27.483362,-97.309704: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.227242
Currently retrieving data for 27.483362,-97.309704: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.124350
Currently retrieving data for 27.483362,-97.309704: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.976456
Currently retrieving data for 27.483362,-97.309704: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:09.834562
Currently retrieving data for 27.483362,-97.309704: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:11.637665
Currently retrieving data for 27.483362,-97.309704: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:13.210755
Currently retrieving data for 27.483362,-97.309704: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:57.375713
Currently retrieving data for 27.483362,-97.309704: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:58.941803
Currently retrieving data for 27.483362,-97.309704: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:02:00.480891
Currently retrieving data for 27.483362,-97.309704: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:02:02.070982
Currently retrieving data for 27.483362,-97.309704: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:02:03.563067
Currently retrieving data for 27.483362,-97.309704: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:02:05.696189
Currently retrieving data for 27.483362,-97.309704: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:07.315282
Currently retrieving data for 27.483362,-97.309704: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:08.623357
Currently retrieving data for 27.483362,-97.309704: from 2017-04-01 to 2017-04-30
Time ela

In [74]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.989393,-122.788328"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.989393,-122.788328


Currently retrieving data for 37.989393,-122.788328: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:02.123122
Currently retrieving data for 37.989393,-122.788328: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.084234
Currently retrieving data for 37.989393,-122.788328: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:06.238357
Currently retrieving data for 37.989393,-122.788328: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:07.947455
Currently retrieving data for 37.989393,-122.788328: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:09.638552
Currently retrieving data for 37.989393,-122.788328: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:11.480657
Currently retrieving data for 37.989393,-122.788328: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:13.113750
Currently retrieving data for 37.989393,-122.788328: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:50.681331
Currently retrieving data for 37.989393,-122.788328: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:52.227419
Currently retrieving data for 37.989393,-122.788328: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:54.099526
Currently retrieving data for 37.989393,-122.788328: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:55.718619
Currently retrieving data for 37.989393,-122.788328: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:57.303710
Currently retrieving data for 37.989393,-122.788328: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:59.097812
Currently retrieving data for 37.989393,-122.788328: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:02:00.396887
Currently retrieving data for 37.989393,-122.788328: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:02:01.764965
Currently retrieving data for 37.989393,-122.788328: from 2017-04-01 to 2017-04-30


In [75]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["38.601800,-77.414362"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 38.601800,-77.414362


Currently retrieving data for 38.601800,-77.414362: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.836105
Currently retrieving data for 38.601800,-77.414362: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.277187
Currently retrieving data for 38.601800,-77.414362: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.600263
Currently retrieving data for 38.601800,-77.414362: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.931339
Currently retrieving data for 38.601800,-77.414362: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.266416
Currently retrieving data for 38.601800,-77.414362: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.970513
Currently retrieving data for 38.601800,-77.414362: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.305589
Currently retrieving data for 38.601800,-77.414362: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:32.579295
Currently retrieving data for 38.601800,-77.414362: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:33.815366
Currently retrieving data for 38.601800,-77.414362: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:35.134441
Currently retrieving data for 38.601800,-77.414362: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.428515
Currently retrieving data for 38.601800,-77.414362: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.758591
Currently retrieving data for 38.601800,-77.414362: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.790708
Currently retrieving data for 38.601800,-77.414362: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.445859
Currently retrieving data for 38.601800,-77.414362: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:43.818938
Currently retrieving data for 38.601800,-77.414362: from 2017-04-01 to 2017-04-30
Time ela

In [76]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["41.795808,-124.086287"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 41.795808,-124.086287


Currently retrieving data for 41.795808,-124.086287: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.444083
Currently retrieving data for 41.795808,-124.086287: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.216184
Currently retrieving data for 41.795808,-124.086287: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.825276
Currently retrieving data for 41.795808,-124.086287: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.536374
Currently retrieving data for 41.795808,-124.086287: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.175468
Currently retrieving data for 41.795808,-124.086287: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.699555
Currently retrieving data for 41.795808,-124.086287: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.207641
Currently retrieving data for 41.795808,-124.086287: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:46.603098
Currently retrieving data for 41.795808,-124.086287: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:48.280194
Currently retrieving data for 41.795808,-124.086287: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:49.839283
Currently retrieving data for 41.795808,-124.086287: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:51.412373
Currently retrieving data for 41.795808,-124.086287: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:53.049466
Currently retrieving data for 41.795808,-124.086287: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:55.452604
Currently retrieving data for 41.795808,-124.086287: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:56.990692
Currently retrieving data for 41.795808,-124.086287: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:58.280766
Currently retrieving data for 41.795808,-124.086287: from 2017-04-01 to 2017-04-30


In [77]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["40.328415,-105.596291"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 40.328415,-105.596291


Currently retrieving data for 40.328415,-105.596291: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.383079
Currently retrieving data for 40.328415,-105.596291: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:04.160238
Currently retrieving data for 40.328415,-105.596291: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.512315
Currently retrieving data for 40.328415,-105.596291: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.822390
Currently retrieving data for 40.328415,-105.596291: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:08.224470
Currently retrieving data for 40.328415,-105.596291: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.727556
Currently retrieving data for 40.328415,-105.596291: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:11.080634
Currently retrieving data for 40.328415,-105.596291: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:37.177558
Currently retrieving data for 40.328415,-105.596291: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:38.561637
Currently retrieving data for 40.328415,-105.596291: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:39.835710
Currently retrieving data for 40.328415,-105.596291: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:41.083782
Currently retrieving data for 40.328415,-105.596291: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:42.393857
Currently retrieving data for 40.328415,-105.596291: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:44.465975
Currently retrieving data for 40.328415,-105.596291: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:46.344083
Currently retrieving data for 40.328415,-105.596291: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:47.832168
Currently retrieving data for 40.328415,-105.596291: from 2017-04-01 to 2017-04-30


In [78]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["32.229252,-110.605921"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 32.229252,-110.605921


Currently retrieving data for 32.229252,-110.605921: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.684097
Currently retrieving data for 32.229252,-110.605921: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.374193
Currently retrieving data for 32.229252,-110.605921: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.830277
Currently retrieving data for 32.229252,-110.605921: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.158353
Currently retrieving data for 32.229252,-110.605921: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.497429
Currently retrieving data for 32.229252,-110.605921: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.827505
Currently retrieving data for 32.229252,-110.605921: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.064576
Currently retrieving data for 32.229252,-110.605921: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:30.182159
Currently retrieving data for 32.229252,-110.605921: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:31.404228
Currently retrieving data for 32.229252,-110.605921: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:32.940316
Currently retrieving data for 32.229252,-110.605921: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:34.219389
Currently retrieving data for 32.229252,-110.605921: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:35.453460
Currently retrieving data for 32.229252,-110.605921: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:37.298566
Currently retrieving data for 32.229252,-110.605921: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:38.527636
Currently retrieving data for 32.229252,-110.605921: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:39.838711
Currently retrieving data for 32.229252,-110.605921: from 2017-04-01 to 2017-04-30


In [79]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["34.111361,-118.926367"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 34.111361,-118.926367


Currently retrieving data for 34.111361,-118.926367: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.401081
Currently retrieving data for 34.111361,-118.926367: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.897166
Currently retrieving data for 34.111361,-118.926367: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.352249
Currently retrieving data for 34.111361,-118.926367: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.671325
Currently retrieving data for 34.111361,-118.926367: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.984400
Currently retrieving data for 34.111361,-118.926367: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.259473
Currently retrieving data for 34.111361,-118.926367: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.634551
Currently retrieving data for 34.111361,-118.926367: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:30.192159
Currently retrieving data for 34.111361,-118.926367: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:31.677244
Currently retrieving data for 34.111361,-118.926367: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:33.187330
Currently retrieving data for 34.111361,-118.926367: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:34.488405
Currently retrieving data for 34.111361,-118.926367: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:35.785479
Currently retrieving data for 34.111361,-118.926367: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.114612
Currently retrieving data for 34.111361,-118.926367: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.015721
Currently retrieving data for 34.111361,-118.926367: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.319796
Currently retrieving data for 34.111361,-118.926367: from 2017-04-01 to 2017-04-30


In [80]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["36.318721,-118.776348"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 36.318721,-118.776348


Currently retrieving data for 36.318721,-118.776348: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.624093
Currently retrieving data for 36.318721,-118.776348: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.169181
Currently retrieving data for 36.318721,-118.776348: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.636265
Currently retrieving data for 36.318721,-118.776348: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.995343
Currently retrieving data for 36.318721,-118.776348: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.302417
Currently retrieving data for 36.318721,-118.776348: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.563490
Currently retrieving data for 36.318721,-118.776348: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.818561
Currently retrieving data for 36.318721,-118.776348: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:31.240218
Currently retrieving data for 36.318721,-118.776348: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.466288
Currently retrieving data for 36.318721,-118.776348: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:33.754362
Currently retrieving data for 36.318721,-118.776348: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.091439
Currently retrieving data for 36.318721,-118.776348: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.427515
Currently retrieving data for 36.318721,-118.776348: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.646642
Currently retrieving data for 36.318721,-118.776348: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.071723
Currently retrieving data for 36.318721,-118.776348: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.466803
Currently retrieving data for 36.318721,-118.776348: from 2017-04-01 to 2017-04-30


In [81]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["38.527962,-78.436940"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 38.527962,-78.436940


Currently retrieving data for 38.527962,-78.436940: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.886108
Currently retrieving data for 38.527962,-78.436940: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.583205
Currently retrieving data for 38.527962,-78.436940: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:05.000286
Currently retrieving data for 38.527962,-78.436940: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:06.351363
Currently retrieving data for 38.527962,-78.436940: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.702440
Currently retrieving data for 38.527962,-78.436940: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.037517
Currently retrieving data for 38.527962,-78.436940: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.384594
Currently retrieving data for 38.527962,-78.436940: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:34.712417
Currently retrieving data for 38.527962,-78.436940: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:36.236504
Currently retrieving data for 38.527962,-78.436940: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:37.522578
Currently retrieving data for 38.527962,-78.436940: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:38.769649
Currently retrieving data for 38.527962,-78.436940: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:40.122727
Currently retrieving data for 38.527962,-78.436940: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:42.254848
Currently retrieving data for 38.527962,-78.436940: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:43.625927
Currently retrieving data for 38.527962,-78.436940: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:44.907000
Currently retrieving data for 38.527962,-78.436940: from 2017-04-01 to 2017-04-30
Time ela

In [82]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["44.715329,-86.116010"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 44.715329,-86.116010


Currently retrieving data for 44.715329,-86.116010: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.578090
Currently retrieving data for 44.715329,-86.116010: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.046174
Currently retrieving data for 44.715329,-86.116010: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.410252
Currently retrieving data for 44.715329,-86.116010: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.803332
Currently retrieving data for 44.715329,-86.116010: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.188411
Currently retrieving data for 44.715329,-86.116010: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.488486
Currently retrieving data for 44.715329,-86.116010: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.768559
Currently retrieving data for 44.715329,-86.116010: from 2011-10-01 to 2011-10-

Time elapsed (hh:mm:ss.ms) 0:01:31.513234
Currently retrieving data for 44.715329,-86.116010: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.893313
Currently retrieving data for 44.715329,-86.116010: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.218389
Currently retrieving data for 44.715329,-86.116010: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.522464
Currently retrieving data for 44.715329,-86.116010: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.895542
Currently retrieving data for 44.715329,-86.116010: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.062666
Currently retrieving data for 44.715329,-86.116010: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.364741
Currently retrieving data for 44.715329,-86.116010: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.768821
Currently retrieving data for 44.715329,-86.116010: from 2017-04-01 to 2017-04-30
Time ela

In [83]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["47.595000,-103.340000"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 47.595000,-103.340000


Currently retrieving data for 47.595000,-103.340000: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.421081
Currently retrieving data for 47.595000,-103.340000: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.975170
Currently retrieving data for 47.595000,-103.340000: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.313246
Currently retrieving data for 47.595000,-103.340000: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.575319
Currently retrieving data for 47.595000,-103.340000: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.861392
Currently retrieving data for 47.595000,-103.340000: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.107463
Currently retrieving data for 47.595000,-103.340000: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.347534
Currently retrieving data for 47.595000,-103.340000: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:34.069380
Currently retrieving data for 47.595000,-103.340000: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:35.368454
Currently retrieving data for 47.595000,-103.340000: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:36.733532
Currently retrieving data for 47.595000,-103.340000: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:38.129612
Currently retrieving data for 47.595000,-103.340000: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:39.460688
Currently retrieving data for 47.595000,-103.340000: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:41.388799
Currently retrieving data for 47.595000,-103.340000: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:42.988890
Currently retrieving data for 47.595000,-103.340000: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:44.271964
Currently retrieving data for 47.595000,-103.340000: from 2017-04-01 to 2017-04-30


In [84]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["40.650613,-122.591211"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 40.650613,-122.591211


Currently retrieving data for 40.650613,-122.591211: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.547089
Currently retrieving data for 40.650613,-122.591211: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.339191
Currently retrieving data for 40.650613,-122.591211: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.661267
Currently retrieving data for 40.650613,-122.591211: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.913338
Currently retrieving data for 40.650613,-122.591211: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.277416
Currently retrieving data for 40.650613,-122.591211: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.566490
Currently retrieving data for 40.650613,-122.591211: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.816562
Currently retrieving data for 40.650613,-122.591211: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:28.426058
Currently retrieving data for 40.650613,-122.591211: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:29.599125
Currently retrieving data for 40.650613,-122.591211: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:30.866197
Currently retrieving data for 40.650613,-122.591211: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:32.133270
Currently retrieving data for 40.650613,-122.591211: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:33.742362
Currently retrieving data for 40.650613,-122.591211: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:35.939487
Currently retrieving data for 40.650613,-122.591211: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:37.224561
Currently retrieving data for 40.650613,-122.591211: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:38.600640
Currently retrieving data for 40.650613,-122.591211: from 2017-04-01 to 2017-04-30


In [85]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["43.565342,-103.489237"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 43.565342,-103.489237


Currently retrieving data for 43.565342,-103.489237: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.543088
Currently retrieving data for 43.565342,-103.489237: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.185182
Currently retrieving data for 43.565342,-103.489237: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.482257
Currently retrieving data for 43.565342,-103.489237: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.892337
Currently retrieving data for 43.565342,-103.489237: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.421425
Currently retrieving data for 43.565342,-103.489237: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.846506
Currently retrieving data for 43.565342,-103.489237: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.250587
Currently retrieving data for 43.565342,-103.489237: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:30.706188
Currently retrieving data for 43.565342,-103.489237: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.005263
Currently retrieving data for 43.565342,-103.489237: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:33.416343
Currently retrieving data for 43.565342,-103.489237: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:34.705417
Currently retrieving data for 43.565342,-103.489237: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.017492
Currently retrieving data for 43.565342,-103.489237: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:37.878599
Currently retrieving data for 43.565342,-103.489237: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:39.173673
Currently retrieving data for 43.565342,-103.489237: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:40.416744
Currently retrieving data for 43.565342,-103.489237: from 2017-04-01 to 2017-04-30


In [86]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["62.529820,-143.258776"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 62.529820,-143.258776


Currently retrieving data for 62.529820,-143.258776: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.668095
Currently retrieving data for 62.529820,-143.258776: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.069171
Currently retrieving data for 62.529820,-143.258776: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.372246
Currently retrieving data for 62.529820,-143.258776: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.703279
Currently retrieving data for 62.529820,-143.258776: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:06.986744
Currently retrieving data for 62.529820,-143.258776: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.326610
Currently retrieving data for 62.529820,-143.258776: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.633684
Currently retrieving data for 62.529820,-143.258776: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:31.258507
Currently retrieving data for 62.529820,-143.258776: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:32.509579
Currently retrieving data for 62.529820,-143.258776: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:33.816654
Currently retrieving data for 62.529820,-143.258776: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:35.111728
Currently retrieving data for 62.529820,-143.258776: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:36.418803
Currently retrieving data for 62.529820,-143.258776: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:38.018894
Currently retrieving data for 62.529820,-143.258776: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:39.404973
Currently retrieving data for 62.529820,-143.258776: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:40.764051
Currently retrieving data for 62.529820,-143.258776: from 2017-04-01 to 2017-04-30


In [87]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["44.973609,-110.693232"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 44.973609,-110.693232


Currently retrieving data for 44.973609,-110.693232: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.641093
Currently retrieving data for 44.973609,-110.693232: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.130179
Currently retrieving data for 44.973609,-110.693232: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.523258
Currently retrieving data for 44.973609,-110.693232: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.852334
Currently retrieving data for 44.973609,-110.693232: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.259415
Currently retrieving data for 44.973609,-110.693232: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:09.038517
Currently retrieving data for 44.973609,-110.693232: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:10.354592
Currently retrieving data for 44.973609,-110.693232: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:32.819309
Currently retrieving data for 44.973609,-110.693232: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:34.128383
Currently retrieving data for 44.973609,-110.693232: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:35.377455
Currently retrieving data for 44.973609,-110.693232: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.635527
Currently retrieving data for 44.973609,-110.693232: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.928601
Currently retrieving data for 44.973609,-110.693232: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.740704
Currently retrieving data for 44.973609,-110.693232: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:41.066780
Currently retrieving data for 44.973609,-110.693232: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:42.513863
Currently retrieving data for 44.973609,-110.693232: from 2017-04-01 to 2017-04-30


In [88]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.798203,-119.866372"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.798203,-119.866372


Currently retrieving data for 37.798203,-119.866372: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.591091
Currently retrieving data for 37.798203,-119.866372: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:03.040174
Currently retrieving data for 37.798203,-119.866372: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.675267
Currently retrieving data for 37.798203,-119.866372: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:09.961569
Currently retrieving data for 37.798203,-119.866372: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:11.281645
Currently retrieving data for 37.798203,-119.866372: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:12.588720
Currently retrieving data for 37.798203,-119.866372: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:14.130808
Currently retrieving data for 37.798203,-119.866372: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:35.801479
Currently retrieving data for 37.798203,-119.866372: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:37.139556
Currently retrieving data for 37.798203,-119.866372: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:38.463631
Currently retrieving data for 37.798203,-119.866372: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:39.750705
Currently retrieving data for 37.798203,-119.866372: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:41.066780
Currently retrieving data for 37.798203,-119.866372: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:42.893885
Currently retrieving data for 37.798203,-119.866372: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:44.197959
Currently retrieving data for 37.798203,-119.866372: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:45.452031
Currently retrieving data for 37.798203,-119.866372: from 2017-04-01 to 2017-04-30


In [89]:
hist_weather_data = retrieve_hist_data("c0214b454c5244a4901191705202706", ["37.204972,-112.982935"], '11-MAR-2011', '11-MAR-2020',24, location_label=False, export_csv=True,
                       store_df=False, response_cache_path=None)



Retrieving weather data for 37.204972,-112.982935


Currently retrieving data for 37.204972,-112.982935: from 2011-03-11 to 2011-03-31
Time elapsed (hh:mm:ss.ms) 0:00:01.380079
Currently retrieving data for 37.204972,-112.982935: from 2011-04-01 to 2011-04-30
Time elapsed (hh:mm:ss.ms) 0:00:02.862163
Currently retrieving data for 37.204972,-112.982935: from 2011-05-01 to 2011-05-31
Time elapsed (hh:mm:ss.ms) 0:00:04.322247
Currently retrieving data for 37.204972,-112.982935: from 2011-06-01 to 2011-06-30
Time elapsed (hh:mm:ss.ms) 0:00:05.688325
Currently retrieving data for 37.204972,-112.982935: from 2011-07-01 to 2011-07-31
Time elapsed (hh:mm:ss.ms) 0:00:07.091405
Currently retrieving data for 37.204972,-112.982935: from 2011-08-01 to 2011-08-31
Time elapsed (hh:mm:ss.ms) 0:00:08.388480
Currently retrieving data for 37.204972,-112.982935: from 2011-09-01 to 2011-09-30
Time elapsed (hh:mm:ss.ms) 0:00:09.714555
Currently retrieving data for 37.204972,-112.982935: from 2011-10-01 to

Time elapsed (hh:mm:ss.ms) 0:01:32.343281
Currently retrieving data for 37.204972,-112.982935: from 2016-09-01 to 2016-09-30
Time elapsed (hh:mm:ss.ms) 0:01:33.609354
Currently retrieving data for 37.204972,-112.982935: from 2016-10-01 to 2016-10-31
Time elapsed (hh:mm:ss.ms) 0:01:34.927429
Currently retrieving data for 37.204972,-112.982935: from 2016-11-01 to 2016-11-30
Time elapsed (hh:mm:ss.ms) 0:01:36.172501
Currently retrieving data for 37.204972,-112.982935: from 2016-12-01 to 2016-12-31
Time elapsed (hh:mm:ss.ms) 0:01:37.338567
Currently retrieving data for 37.204972,-112.982935: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:01:39.014663
Currently retrieving data for 37.204972,-112.982935: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:01:40.241733
Currently retrieving data for 37.204972,-112.982935: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:01:41.532807
Currently retrieving data for 37.204972,-112.982935: from 2017-04-01 to 2017-04-30
